In [ ]:
import warnings
warnings.filterwarnings('ignore')

import librosa
import os

In [ ]:
TRAIN_PATH = 'data/train/' # File path for training data
TEST_PATH = 'data/test/' # File path for Test data
VAL_PATH = 'data/val/' # File path where you upload your own recording to check

speech_cats =['down', 'up'] # We process only two categories - Up and Down

In [ ]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Unzip the files
!unzip drive/MyDrive/'PGP NLP'/train.zip -d ./data/
!unzip drive/MyDrive/'PGP NLP'/test.zip -d ./data/

In [ ]:
# Sample file
x,sr = librosa.load('data/train/down/030ec18b_nohash_0.wav')

In [ ]:
import IPython.display as ipd

ipd.Audio('data/train/down/030ec18b_nohash_0.wav')

In [ ]:
mfcc = librosa.feature.mfcc(x,n_mfcc=13, hop_length = 512,sr=22050, n_fft=2048)

In [ ]:
mfcc.shape

In [ ]:
# Function to process data.
# I/P : Path to folders
# Output is a Dictionary with two lists - one for labels and one for MFCC
def prep_data(path):
    data = {'label': [],'mfcc': []}
    for x in speech_cats:
        print(path+x)
        for fn in os.listdir(path+"/"+x+"/"):
            signal, sr = librosa.load(path+"/"+x+"/"+fn)
            if len(signal) >= 22050:
                mfcc = librosa.feature.mfcc(signal[:22050], sr=sr, n_mfcc=13, hop_length = 512, n_fft=2048)
                
                data['label'].append(x)
                data['mfcc'].append(mfcc.T.tolist())
    return(data)

In [ ]:
# Prepare the Test and Train data
test = prep_data(TEST_PATH)
train = prep_data(TRAIN_PATH)

In [ ]:
# Import Numpy and prepare the X and y variables
import numpy as np
X_train = np.array(train['mfcc'])
X_test = np.array(test['mfcc'])
y_train = np.array(train['label'])
y_test = np.array(test['label'])

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
np.unique(y_train)

In [ ]:
# Label Encoding for the y variables
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train_c = le.fit_transform(y_train)
y_test_c = le.transform(y_test)

In [ ]:
le.classes_

In [ ]:
le.transform(['up'])

In [ ]:
# Import relevant components from Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout

In [ ]:
# Instantiate a model
model = Sequential()

In [ ]:
# First Conv layer with Maxpool
model.add(Conv2D(128, 3, activation = 'relu',padding='same', input_shape=(X_train.shape[1], X_train.shape[2],1)))
model.add(MaxPool2D(2))

# Second Conv layer with Maxpool
model.add(Conv2D(128, 3, activation = 'relu', padding='same'))
model.add(MaxPool2D(2))

# Third Conv layer with Maxpool
model.add(Conv2D(128, 3, activation = 'relu', padding='same'))
model.add(MaxPool2D(2))

# Flatten layer
model.add(Flatten())

# Dense Layer
model.add(Dense(64, activation = 'relu'))

# Output layer
model.add(Dense(1, activation = 'sigmoid'))

In [ ]:
# Compile Model
model.compile(optimizer = 'adam', loss=tf.keras.losses.BinaryCrossentropy(), metrics = ['accuracy'])

In [ ]:
# Fit the model
model.fit(X_train, y_train_c, epochs = 10, validation_data = (X_test, y_test_c))

In [ ]:
# Since the self recorded files have been appropriately uploaded
# Prepare the validation data
val = prep_data(VAL_PATH)

In [ ]:
X_val = np.array(val['mfcc'])

In [ ]:
# Predict the validation data
model.predict(X_val)>0.5

In [ ]:
val['label']

In [ ]:
le.classes_

In [ ]:
le.transform(['up'])